In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)


In [6]:
print(tokenizer.default_chat_template)

{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alter

In [19]:
from datasets import load_dataset

system_prompt = """Create a text ad given the following product and description."""

def preprocess(samples):
    batch = []
    for product, desc, ad_copy in zip(samples["product"],samples["description"],samples["ad"]):
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"""Product: {product}\nDescription: {desc}\n"""},
            {"role": "assistant", "content": f"""Ad: {ad_copy}\n"""},
        ]
        batch.append(tokenizer.apply_chat_template(conversation, tokenize=False))
    return {"content": batch}
            
    


dataset = load_dataset("jaykin01/advertisement-copy")
dataset
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

dataset["train"] = dataset["train"].shuffle(100)
dataset_subsets = dataset["train"].train_test_split(141)
dataset["train"] = dataset_subsets["train"]
dataset["test"] = dataset_subsets["test"]
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['content'],
        num_rows: 141
    })
})
{'content': '<s>[INST] <<SYS>>\nCreate a text ad given the following product and description.\n<</SYS>>\n\nProduct: Fitness Magazine\nDescription: Fitness magazine for staying active and achieving your fitness goals. [/INST] Ad: Stay active with a Fitness Magazine! 💪📖 Experience fitness tips and motivating stories. Perfect for fitness enthusiasts and reaching your health and wellness goals. Limited stock - achieve fitness with a touch of motivation! 🌟🌟🏋️\u200d♀️ </s>'}


In [20]:
dataset.push_to_hub("ad-copy-generation", private=False)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]